In [15]:
import pandas as pd
import numpy as np
import soundfile as sf
import warnings

In [16]:
raw_metadata_path = 'orcasound/orcasound_reformated.csv'
splited_metadata_path = 'orcasound/orcasound_train_val_splits.csv'

metadata = pd.read_csv(raw_metadata_path)

In [17]:
def calc_sum_column(df, label, column_name='call_length'):
    return sum(df[df['label']==label][column_name])

def add_train_val_split(df, add_new_column=True, val_ratio=0.1):
    assert val_ratio < 1, 'val ratio should be smaller than 1'
    if add_new_column:
        if 'split_type' in df.columns:
            warnings.warn('trying to update existing split, abort!')
            return df
        df['split_type'] = ""
    for label in (0, 1):
        total_len = calc_sum_column(df, label, 'call_length')
        total_val = 0
        for it in df[df['label']==label].index:
            df.at[it, 'split_type'] = 'val'
            total_val += df.at[it, 'call_length']
            if total_val >= total_len * val_ratio:
                break
        df.at[(df['label']==label) & (df['split_type']==''), 'split_type'] = 'train'
    return df



In [18]:
metadata = add_train_val_split(metadata)

In [19]:
metadata.to_csv(splited_metadata_path, index=False)